# Convert Wingtra flight plan to KML

In [6]:
# Author: Farid Javadnejad
# Date: 2025-02-18
# Last Update: 2025-6-11
#
# DESCRIPTION:
# This script converts a Wingtra UAS flight plan from a .flightplan file (i.e., JSON) to a KML file, 
# extracting the survey area and adding a Placemark for the takeoff/landing location.
#
# DISCLAIMER:
# This script was developed with the assistance of AI tools for debugging, reviewing, and testing.
# ---------------------------------------------------------------------------------------------------

In [ ]:
#Install

#!pip install simplekml fastkml lxml


In [8]:
#pip install simplekml fastkml lxml
import json
import os
import simplekml

In [9]:
def read_flightplan(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

def extract_survey_areas(flightplan):
    areas = []
    for item in flightplan.get("flightPlan", {}).get("items", []):
        if item.get("complexItemType") == "area" and "polygon" in item:
            areas.append(item["polygon"])
    return areas

def extract_corridors(flightplan):
    corridors = []
    for item in flightplan.get("flightPlan", {}).get("items", []):
        if item.get("complexItemType") == "corridor" and "corridor" in item:
            polyline = item["corridor"].get("polyline", [])
            if polyline:
                corridors.append(polyline)
    return corridors

def convert_to_kml_coordinates(coords):
    try:
        return [(point[1], point[0]) for point in coords]
    except IndexError:
        print("Invalid coordinate format.")
        return None

def add_flightplan_features(flights_folder, takeoff_folder, flightplan_data, base_name, fill_color):
    areas = extract_survey_areas(flightplan_data)
    corridors = extract_corridors(flightplan_data)
    takeoff_landing = flightplan_data.get("flightPlan", {}).get("items", [])[0].get("coordinate", [])

    # Add each area
    for idx, area in enumerate(areas, start=1):
        coords = convert_to_kml_coordinates(area)
        if coords:
            pol = flights_folder.newpolygon(name=f"{base_name} - Area {idx}", outerboundaryis=coords)
            pol.style.polystyle.color = simplekml.Color.rgb(fill_color[0], fill_color[1], fill_color[2], 128)
            pol.style.linestyle.color = simplekml.Color.rgb(255, 255, 255)
            pol.style.linestyle.width = 1.0

    # Add each corridor
    for idx, corridor in enumerate(corridors, start=1):
        coords = convert_to_kml_coordinates(corridor)
        if coords:
            line = flights_folder.newlinestring(name=f"{base_name} - Corridor {idx}", coords=coords)
            line.style.linestyle.color = simplekml.Color.rgb(fill_color[0], fill_color[1], fill_color[2], 128)
            line.style.linestyle.width = 5.0

    # Add takeoff/landing point
    if takeoff_landing:
        pt = takeoff_folder.newpoint(name=f"{base_name} - H", coords=[(takeoff_landing[1], takeoff_landing[0], takeoff_landing[2])])
        pt.style.iconstyle.icon.href = "https://maps.google.com/mapfiles/kml/shapes/target.png"
        pt.style.iconstyle.scale = 0.9
        pt.style.iconstyle.color = simplekml.Color.rgb(0, 170, 255)
        pt.style.labelstyle.scale = 0.7
        pt.style.labelstyle.color = simplekml.Color.rgb(0, 170, 255)



In [10]:
# Define color palette
Blue = (0,170,255) 
Green = (0,255,0) 
Orange = (255,170,0) 
Magenta = (255,0,255)
Red = (255, 0, 0)
Yellow = (255, 255, 0)
Cyan = (0, 255, 255)
Purple = (128, 0, 128)

colors = [Blue, Green, Orange, Magenta, Red, Yellow, Cyan, Purple]

# Input/Output paths
read_folder = r"C:\Users\USFJ139860\WSP O365\SW Geomatics Projects - US-WSP-30902453.P4 CN 5101175 - US64 Imagery\RS\00_PLANNING\02_FLIGHT_PLANS"
read_file_extension = ".flightplan"
output_kml_path = os.path.join(read_folder, "combined_flightplans.kml")

# Create the main KML and folders
kml = simplekml.Kml()
flights_folder = kml.newfolder(name="Flights")
takeoff_folder = kml.newfolder(name="Takeoff/Land")

# Loop through files and build KML content
index = 0
for read_file in os.listdir(read_folder):
    if read_file.endswith(read_file_extension):
        file_path = os.path.join(read_folder, read_file)
        flightplan = read_flightplan(file_path)

        fill_color = colors[index]
        index = (index + 1) % len(colors)

        base_name = os.path.splitext(read_file)[0]
        add_flightplan_features(flights_folder, takeoff_folder, flightplan, base_name, fill_color)

# Save the combined KML
kml.save(output_kml_path)
print(f"KML saved at: {output_kml_path}")


KML saved at: C:\Users\USFJ139860\WSP O365\SW Geomatics Projects - US-WSP-30902453.P4 CN 5101175 - US64 Imagery\RS\00_PLANNING\02_FLIGHT_PLANS\combined_flightplans.kml
